<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg" />
</center> 
     

## <center> Kaggle inclass competition from [mlcourse.ai](https://mlcourse.ai/)
    
# <center> [**Catch me if you can**](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2)

### <center> Session: Fall 2019

#### <div style="text-align: right"> Author: [Vladimir Kulyashov](https://github.com/koolvn)


<div style="text-align: right"> creation date: 15 October 2019 </div>

**Prerequisitions proved by EDA:**
1. Alice lives in France
2. Data was collected in the university at working hours 
3. Alice used PC mostly for watching videos and social networks
4. Alice doesn't use GMail or Google+ and Bing


**Goal:**
Beat the A3 strong baseline (0.95965) baseline with as less features, as possible

In [103]:
# Import libraries and set desired options
import os
import pickle
import numpy as np
import pandas as pd
from scipy.sparse import hstack
# !pip install eli5
import eli5
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display_html

In [104]:
PATH_TO_DATA = '../data/alice/'
filename = f'submission_1.csv'
pred_path = './predictions/alice/'
SEED = 17
time_split = TimeSeriesSplit(n_splits=10)
logit = LogisticRegression(C=1, random_state=SEED, solver='liblinear')
BEST_LOGIT_C = 2.5118864315095824
BEST_LOGIT_TOL = 0.004

In [105]:
def prepare_sparse_features(path_to_train, path_to_test, path_to_site_dict,
                           vectorizer_params):
    """ Prepares sparsed X_train, X_test, y_train, vectorizer, train_times, test_times,
        train_sites, test_sites, top_alice_sites
        
        from input CSV files, pickle file and vectorizer_params dictionary.
    
        return:: X_train, X_test, y_train, vectorizer, train_times, test_times, train_sites, test_sites, top_alice_sites """
    
    times = ['time%s' % i for i in range(1, 11)]
    train_df = pd.read_csv(path_to_train,
                       index_col='session_id', parse_dates=times)
    test_df = pd.read_csv(path_to_test,
                      index_col='session_id', parse_dates=times)

    # Sort the data by time
    train_df = train_df.sort_values(by='time1')
    
    # read site -> id mapping provided by competition organizers 
    with open(path_to_site_dict, 'rb') as f:
        site2id = pickle.load(f)
    # create an inverse id _> site mapping
    id2site = {v:k for (k, v) in site2id.items()}
    # we treat site with id 0 as "unknown"
    id2site[0] = 'unknown'
    
    # Transform data into format which can be fed into TfidfVectorizer
    # This time we prefer to represent sessions with site names, not site ids. 
    # It's less efficient but thus it'll be more convenient to interpret model weights.
    sites = ['site%s' % i for i in range(1, 11)]
    train_sessions = train_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    test_sessions = test_df[sites].fillna(0).astype('int').apply(lambda row: 
                                                     ' '.join([id2site[i] for i in row]), axis=1).tolist()
    
    sites_dict = pd.DataFrame(list(site2id.keys()),
                              index=list(site2id.values()),
                              columns=['site'])
    
    top_alice_sites = pd.Series(train_df[train_df['target'] == 1][sites].fillna(0).astype('int').values.flatten()
                               ).value_counts().sort_values(ascending=False).head(1)
    # we'll tell TfidfVectorizer that we'd like to split data by whitespaces only 
    # so that it doesn't split by dots (we wouldn't like to have 'mail.google.com' 
    # to be split into 'mail', 'google' and 'com')
    vectorizer = TfidfVectorizer(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_sessions)
    X_test = vectorizer.transform(test_sessions)
    y_train = train_df['target'].astype('int').values
    
    # we'll need site visit times for further feature engineering
    train_times, test_times = train_df[times], test_df[times]
    
    # sites_df
    train_sites, test_sites = train_df[sites].fillna(0).astype('int'), test_df[sites].fillna(0).astype('int')
    
    full_df = pd.concat([train_df.drop('target', axis=1), test_df])
    
    return X_train, X_test, y_train, vectorizer, train_times, test_times, train_sites, test_sites, top_alice_sites

In [106]:
%%time
X_train_sites, X_test_sites, y_train, vectorizer, train_times, test_times, train_sites, test_sites, top_alice_sites = prepare_sparse_features(
    path_to_train=os.path.join(PATH_TO_DATA, 'train_sessions.csv'),
    path_to_test=os.path.join(PATH_TO_DATA, 'test_sessions.csv'),
    path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl'),
    vectorizer_params={'ngram_range': (1, 5), 
                       'max_features': 50000,
                       'tokenizer': lambda s: s.split()}
)


Wall time: 36.9 s


In [107]:
# A helper function for writing predictions to a file and write list of features of that predictions
def write_to_submission_file(predicted_labels, out_file, new_feature_names=None,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)
    
    text_file = open(out_file+'.txt', "w")
    text_file.write(str(new_feature_names))
    text_file.close()

# I'm lazy, so here is a function that names files for you    
def get_next_filename(filename='submission_1.csv', path=pred_path, file_exists=False, i=1):
    if filename in os.listdir(path):
        file_exists = True
        while file_exists:
            i += 1
            next_ = list(filename.split('.')[0].split('_')[0])
            next_.append('_')
            next_.append(str(i))
            next_ = ''.join(next_) + '.csv'
            next_, file_exists = get_next_filename(next_, path, False, i)
            
        return next_, file_exists  
    else:
        file_exist = False
        return filename, file_exists
    
def train_and_predict(model, X_train, y_train, X_test, site_feature_names=vectorizer.get_feature_names(), 
                      new_feature_names=None, cv=time_split, scoring='roc_auc',
                      top_n_features_to_show=30, submission_file_name='submission.csv'):
    
    
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, 
                            scoring=scoring, n_jobs=4)
    print('CV scores', cv_scores)
    print('\nCV mean: {}, CV std: {}'.format(cv_scores.mean(), cv_scores.std()))
    model.fit(X_train, y_train)
    
    if new_feature_names:
        all_feature_names = site_feature_names + new_feature_names 
    else: 
        all_feature_names = site_feature_names
    
    display_html(eli5.show_weights(estimator=model, 
                  feature_names=all_feature_names, top=top_n_features_to_show))
    
    if new_feature_names:
        print('New feature weights:')
    
        print(pd.DataFrame({'feature': new_feature_names, 
                        'coef': model.coef_.flatten()[-len(new_feature_names):]}).sort_values(by='coef'))
    
    test_pred = model.predict_proba(X_test)[:, 1]
    write_to_submission_file(test_pred, submission_file_name, new_feature_names) 
    
    return cv_scores

### Base score with no features added

In [108]:
%%time
cv_scores_base = train_and_predict(model=logit,
                               X_train=X_train_sites,
                               y_train=y_train,
                               X_test=X_test_sites,
                               site_feature_names=vectorizer.get_feature_names(),
                               cv=time_split, submission_file_name=pred_path+'base_'+filename)

CV scores [0.83124023 0.65993466 0.85673565 0.92824237 0.84779639 0.88954524
 0.88829128 0.8771044  0.92023038 0.92624125]

CV mean: 0.8625361862232206, CV std: 0.0745567081201155


Weight?,Feature
+5.880,youwatch.org
+5.380,cid-ed6c3e6a5c6608a4.users.storage.live.com
+5.222,fr.glee.wikia.com
+5.114,vk.com
+4.875,www.info-jeunes.net
+4.499,www.banque-chalus.fr
+4.220,www.express.co.uk
+4.147,www.audienceinsights.net
+4.089,www.melty.fr
+4.003,glee.hypnoweb.net


Wall time: 12.8 s


-----------------------------------------

### Adding features

### Feature engineering fields

In [109]:
with open(PATH_TO_DATA + 'site_dic.pkl', "rb") as input_file:
        site_dict = pickle.load(input_file)
        
facebook_ids = []
youtube_ids = []
google_ids = []
france_ids = []
sowftware_ids = []
news_ids = []

for key in list(site_dict.keys()):
    if 'facebook' in key:
        facebook_ids.append(site_dict[key])
    if 'youtube' in key or 'youwatch' in key:
        youtube_ids.append(site_dict[key])
    if 'google' in key:
        google_ids.append(site_dict[key])
    if '.fr' in key:
        france_ids.append(site_dict[key])
    if 'browser-update.org' in key:
        sowftware_ids.append(site_dict[key])
    if 'express.co.uk' in key:
        news_ids.append(site_dict[key])
            
sites_dict = pd.DataFrame(list(site_dict.keys()),
                          index=list(site_dict.values()),
                          columns=['site'])

scaler = StandardScaler()


In [110]:
alice = train_df[train_df['target']==1]
alice[['site1','site2','site3']].nunique()

site1    448
site2    436
site3    446
dtype: int64

In [111]:
times = ['time%s' % i for i in range(1, 11)]
sites = ['site%s' % i for i in range(1, 11)]

train_df = pd.read_csv(PATH_TO_DATA + 'train_sessions.csv',
                       index_col='session_id', parse_dates=times)

top_alice_sites = pd.Series(train_df[train_df['target'] == 1][sites].fillna(0).astype('int').values.flatten()
                               ).value_counts().sort_values(ascending=False).head(3)

In [112]:
train_df[train_df['target'] == 1][sites].fillna(0).astype('int').values.flatten()

array([5397, 5395,   22, ...,  879,   80,  879])

In [113]:
(train_times['time1'].dt.month == 5).astype('int').value_counts()

0    250332
1      3229
Name: time1, dtype: int64

In [114]:
((train_times['time1'].dt.month == 10) | (train_times['time1'].dt.month == 5)).astype('int').value_counts()

0    247544
1      6017
Name: time1, dtype: int64

In [115]:
def add_features(times, sites, X_sparse, top_alice_sites):
    
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    
    with open(PATH_TO_DATA + 'site_dic.pkl', "rb") as input_file:
        site_dict = pickle.load(input_file)
        
        
    sites_dict = pd.DataFrame(list(site_dict.keys()),
                              index=list(site_dict.values()),
                              columns=['site'])
        
    # time features
    hour = times['time1'].dt.hour
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <=6)).astype('int').values.reshape(-1, 1)
    
    durations = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[ms]').astype('int').values.reshape(-1, 1)
    durations = scaler.fit_transform(durations)
    
    week = times['time1'].dt.week.values.reshape(-1, 1)
    week = scaler.fit_transform(week)

    winter = times['time1'].dt.month.isin([12, 1, 2]).astype('int').values.reshape(-1, 1)
    spring = times['time1'].dt.month.isin([3, 4, 5]).astype('int').values.reshape(-1, 1)
    summer = times['time1'].dt.month.isin([6, 7, 8]).astype('int').values.reshape(-1, 1)
    autumn = times['time1'].dt.month.isin([9, 10, 11]).astype('int').values.reshape(-1, 1)
    
    day_of_week = times['time1'].dt.weekday.astype('int').values.reshape(-1, 1)
    month = times['time1'].dt.month.astype('int').values.reshape(-1, 1)
    year_month = times['time1'].apply(lambda ts: 100 * ts.year + ts.month).astype('int').values.reshape(-1, 1)
    year_month = scaler.fit_transform(year_month)
    
    sunday = (times['time1'].dt.weekday == 6).astype('int').values.reshape(-1, 1)
    
    may = (times['time1'].dt.month == 5).astype('int').values.reshape(-1, 1)
    october = (times['time1'].dt.month == 10).astype('int').values.reshape(-1, 1)
    
    # site features
    facebook_ids = []
    youtube_ids = []
    google_ids = []
    france_ids = []
    vk_ids = []
    msft_ids = []
    bing_ids = []

    for key in list(site_dict.keys()):
        if 'facebook' in key:
            facebook_ids.append(site_dict[key])
        if 'youtube' in key  in key or 'video' in key:# or 'ytimg.com' in key:
            youtube_ids.append(site_dict[key])
        if 'mail.google.com' in key or 'plus.google.com' in key: # or 'wwww.bing.com' in key:
            google_ids.append(site_dict[key])
        if '.fr' in key or 'fr.' in key:
            france_ids.append(site_dict[key])
        if 'vk.com' in key or 'vk.ru' in key:
            vk_ids.append(site_dict[key])
        if 'storage.live' in key:
            msft_ids.append(site_dict[key])
        if 'bing.com' in key:
            bing_ids.append(site_dict[key])
            
        
    sites_dict.loc[top_alice_sites.index]
    top_alice_ids = []

    for key in top_alice_sites.index:
        top_alice_ids.append(key)

    in_alice_top = sites.isin(top_alice_sites).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_google = sites['site1'].isin(google_ids).astype('int').values.reshape(-1, 1)
    has_google = sites.isin(google_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_youtube = sites['site1'].isin(youtube_ids).astype('int').values.reshape(-1, 1)
    has_youtube = sites.isin(youtube_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_facebook = sites['site1'].isin(facebook_ids).astype('int').values.reshape(-1, 1)
    has_facebook = sites.isin(facebook_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_vk = sites['site1'].isin(vk_ids).astype('int').values.reshape(-1, 1)
    has_vk = sites.isin(vk_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).values.reshape(-1, 1)
    fr_domains = scaler.fit_transform(fr_domains)
#     fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    msft_usage = sites.isin(msft_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_bing = sites['site1'].isin(bing_ids).astype('int').values.reshape(-1, 1)
    
    # stacking matrix
    objects_to_hstack = [X_sparse, morning, day, evening, night, durations,
                         day_of_week,
                         year_month,
                         sunday,
                         start_google,
                         start_youtube,
                         fr_domains,
                         start_vk,
                         msft_usage,
                         summer,
#                          week,
                         has_google,
                         has_youtube,
#                          in_alice_top,
                         has_vk,
                         has_facebook,
                         may,
                         october,
                        ]
    
    feature_names = ['morning', 'day','evening', 'night', 'durations',
                     'day_of_week',
                     'year_month',
                     'sunday',
                     'start_google',
                     'start_youtube',
                     'fr_domains',
                     'start_vk',
                     'msft_usage', 
                     'summer',
#                      'week',
                     'has_google',
                     'has_youtube',
#                      'in_alice_top',
                     'has_vk',
                     'has_facebook',
                     'may',
                     'october',
                    ]
        
    X = hstack(objects_to_hstack)
    return X, feature_names

### Training model with new features

In [116]:
%%time
X_train, new_feat_names = add_features(train_times, train_sites, X_train_sites, top_alice_sites)
X_test, _ = add_features(test_times, test_sites, X_test_sites, top_alice_sites)

Wall time: 4.18 s


In [117]:
new_feat_names

['morning',
 'day',
 'evening',
 'night',
 'durations',
 'day_of_week',
 'year_month',
 'sunday',
 'start_google',
 'start_youtube',
 'fr_domains',
 'start_vk',
 'msft_usage',
 'summer',
 'has_google',
 'has_youtube',
 'has_vk',
 'has_facebook',
 'may',
 'october']

In [118]:
%%time
cv_scores_engineered = train_and_predict(model=logit, X_train=X_train, y_train=y_train,
                                         X_test=X_test, 
                                         site_feature_names=vectorizer.get_feature_names(),
                                         new_feature_names=new_feat_names,
                                         cv=time_split,
                                         submission_file_name=pred_path + 'engineered_' + filename)

CV scores [0.83393992 0.85303472 0.95070977 0.96547697 0.92043363 0.95368435
 0.93260334 0.94515324 0.96448755 0.9692297 ]

CV mean: 0.9288753172476136, CV std: 0.04521424131811921


Weight?,Feature
+4.978,www.express.co.uk
+4.903,cid-ed6c3e6a5c6608a4.users.storage.live.com
+4.785,www.info-jeunes.net
+4.702,youwatch.org
+4.383,www.audienceinsights.net
+4.369,fr.glee.wikia.com
+4.051,www.melty.fr
+3.987,api.bing.com
+3.877,www.banque-chalus.fr
+3.777,www.kelbillet.com


New feature weights:
          feature      coef
14     has_google -3.967366
0         morning -3.113359
2         evening -2.621278
13         summer -2.536448
7          sunday -1.703323
19        october -1.702946
18            may -0.893361
8    start_google -0.486496
6      year_month -0.483409
5     day_of_week -0.355590
4       durations -0.228219
3           night  0.000000
11       start_vk  0.078936
10     fr_domains  0.173435
9   start_youtube  0.196394
12     msft_usage  0.441572
17   has_facebook  0.463546
1             day  0.617811
15    has_youtube  0.639359
16         has_vk  1.839730
Wall time: 21.3 s


In [119]:
cv_scores_base < cv_scores_engineered

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

Leaderboard score = 0.96215

### Parameter tuning

In [125]:
%%time
logit = LogisticRegression(random_state=SEED, solver='liblinear')

params = {'C': np.logspace(0, 1.05, 10), 'tol': [BEST_LOGIT_TOL]}#np.linspace(0.1, 0.001, 10)}

# params = {'C': [BEST_LOGIT_C],
#           'tol': [BEST_LOGIT_TOL]}


logit_grid_searcher = GridSearchCV(estimator=logit, param_grid=params,
                              scoring='roc_auc', n_jobs=4, cv=time_split, verbose=1)

logit_grid_searcher.fit(X_train, y_train)

print('Tuned score', logit_grid_searcher.best_score_)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   42.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  2.2min finished


Tuned score 0.9310194523889457
Wall time: 2min 21s


In [126]:
logit_grid_searcher.best_estimator_

LogisticRegression(C=2.2387211385683394, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=17, solver='liblinear', tol=0.004, verbose=0,
                   warm_start=False)

### Training the best model and writing the output files:
submission.csv + text file with features and best parameters

In [127]:
%%time
cv_scores_tuned = train_and_predict(model=logit_grid_searcher.best_estimator_, X_train=X_train, y_train=y_train,
                                         X_test=X_test, 
                                         site_feature_names=vectorizer.get_feature_names(),
                                         new_feature_names=new_feat_names,
                                         cv=time_split,
                                         submission_file_name=pred_path + get_next_filename(filename)[0])

CV scores [0.84745578 0.84424754 0.95372735 0.96891364 0.92485909 0.95448724
 0.93823836 0.94358888 0.96634625 0.96833039]

CV mean: 0.9310194523889457, CV std: 0.04461249710140548


Weight?,Feature
+8.969,www.express.co.uk
+7.917,cid-ed6c3e6a5c6608a4.users.storage.live.com
+5.746,browser-update.org
+5.687,tru.am
+5.410,fr.glee.wikia.com
+5.309,www.info-jeunes.net
+5.208,api.bing.com
+5.164,youwatch.org
+5.029,www.banque-chalus.fr
+4.963,www.kelbillet.com


New feature weights:
          feature      coef
14     has_google -4.542586
13         summer -3.174974
2         evening -3.118442
0         morning -3.112895
19        october -2.223519
7          sunday -1.967759
18            may -1.168111
6      year_month -0.532270
8    start_google -0.528712
5     day_of_week -0.356959
4       durations -0.210295
3           night  0.000000
11       start_vk  0.010712
10     fr_domains  0.156372
9   start_youtube  0.162979
12     msft_usage  0.323044
17   has_facebook  0.406575
15    has_youtube  0.502766
1             day  0.695514
16         has_vk  1.824699
Wall time: 20.3 s


In [128]:
cv_scores_engineered < cv_scores_tuned

array([ True, False,  True,  True,  True,  True,  True, False,  True,
       False])

In [21]:
# working hours feature + mean the latest 3 submissions + weekend feature + mail feature

Leaderboard score: **0.96227**

http://browser-update.org/ - looks like Alice has a 3rd party app installed.
www.express.co.uk - Alice reads news often

Let's find out.

In [99]:
def add_features(times, sites, X_sparse, top_alice_sites):
    
    scaler = StandardScaler()
#     scaler = MinMaxScaler()
    
    with open(PATH_TO_DATA + 'site_dic.pkl', "rb") as input_file:
        site_dict = pickle.load(input_file)
        
        
    sites_dict = pd.DataFrame(list(site_dict.keys()),
                              index=list(site_dict.values()),
                              columns=['site'])
        
    # time features
    hour = times['time1'].dt.hour
    morning = ((hour >= 7) & (hour <= 11)).astype('int').values.reshape(-1, 1)
    day = ((hour >= 12) & (hour <= 18)).astype('int').values.reshape(-1, 1)
    evening = ((hour >= 19) & (hour <= 23)).astype('int').values.reshape(-1, 1)
    night = ((hour >= 0) & (hour <=6)).astype('int').values.reshape(-1, 1)
    
    durations = (times.max(axis=1) - times.min(axis=1)).astype('timedelta64[ms]').astype('int').values.reshape(-1, 1)
    durations = scaler.fit_transform(durations)
    
    week = times['time1'].dt.week.values.reshape(-1, 1)
    week = scaler.fit_transform(week)

    winter = times['time1'].dt.month.isin([12, 1, 2]).astype('int').values.reshape(-1, 1)
    spring = times['time1'].dt.month.isin([3, 4, 5]).astype('int').values.reshape(-1, 1)
    summer = times['time1'].dt.month.isin([6, 7, 8]).astype('int').values.reshape(-1, 1)
    autumn = times['time1'].dt.month.isin([9, 10, 11]).astype('int').values.reshape(-1, 1)
    
    day_of_week = times['time1'].dt.weekday.astype('int').values.reshape(-1, 1)
    month = times['time1'].dt.month.astype('int').values.reshape(-1, 1)
    year_month = times['time1'].apply(lambda ts: 100 * ts.year + ts.month).astype('int').values.reshape(-1, 1)
    year_month = scaler.fit_transform(year_month)
    
    sunday = (times['time1'].dt.weekday == 6).astype('int').values.reshape(-1, 1)
    wednesday = (times['time1'].dt.weekday == 5).astype('int').values.reshape(-1, 1)

    may = (times['time1'].dt.month == 5).astype('int').values.reshape(-1, 1)
    october = (times['time1'].dt.month == 10).astype('int').values.reshape(-1, 1)
    
    
    # site features
    facebook_ids = []
    youtube_ids = []
    google_ids = []
    france_ids = []
    vk_ids = []
    msft_ids = []
    bing_ids = []
    software_ids = []
    alice_top_ids = []

    for key in list(site_dict.keys()):
        if 'facebook' in key:
            facebook_ids.append(site_dict[key])
        if 'youtube' in key  in key or 'video' in key:# or 'ytimg.com' in key:
            youtube_ids.append(site_dict[key])
        if 'mail.google.com' in key or 'plus.google.com' in key: # or 'wwww.bing.com' in key:
            google_ids.append(site_dict[key])
        if '.fr' in key or 'fr.' in key:
            france_ids.append(site_dict[key])
        if 'vk.com' in key or 'vk.ru' in key:
            vk_ids.append(site_dict[key])
        if 'storage.live' in key:
            msft_ids.append(site_dict[key])
        if 'bing.com' in key:
            bing_ids.append(site_dict[key])
        if 'news' in key or 'mail' in key or 'express' in key:
            software_ids.append(site_dict[key])
        
        if 'www.google.fr' in key or 'www.youtube.com' in key or 's.youtube.com' in key or 'www.facebook.com' in key or 'i1.ytimg.com' in key:
            alice_top_ids.append(site_dict[key])
            
        
    sites_dict.loc[top_alice_sites.index]
    top_alice_ids = []

    for key in top_alice_sites.index:
        top_alice_ids.append(key)

    in_alice_top = sites.isin(top_alice_sites).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
    start_google = sites['site1'].isin(google_ids).astype('int').values.reshape(-1, 1)
    has_google = sites.isin(google_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_youtube = sites['site1'].isin(youtube_ids).astype('int').values.reshape(-1, 1)
    has_youtube = sites.isin(youtube_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_facebook = sites['site1'].isin(facebook_ids).astype('int').values.reshape(-1, 1)
    has_facebook = sites.isin(facebook_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    start_vk = sites['site1'].isin(vk_ids).astype('int').values.reshape(-1, 1)
    has_vk = sites.isin(vk_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).values.reshape(-1, 1)
    fr_domains = scaler.fit_transform(fr_domains)
#     fr_domains = sites.isin(france_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    msft_usage = sites.isin(msft_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    
#     start_bing = sites['site1'].isin(bing_ids).astype('int').values.reshape(-1, 1)
    has_bing = sites.isin(bing_ids).astype('int').astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    has_software = sites.isin(sowftware_ids).astype('int').sum(axis=1).astype('bool').astype('int').values.reshape(-1, 1)
    has_alice_top = sites.isin(top_alice_ids).astype('int').sum(axis=1).values.reshape(-1, 1)
    has_alice_top = scaler.fit_transform(has_alice_top)
    
    # stacking matrix
    objects_to_hstack = [X_sparse, morning, day, evening, night,
#                          durations,
                         day_of_week,
                         year_month,
                         sunday,
                         start_google,
                         start_youtube,
                         fr_domains,
                         start_vk,
                         msft_usage,
                         summer,
#                          week,
                         has_google,
                         has_youtube,
#                          in_alice_top,
                         has_vk,
                         has_facebook,
                         may,
                         october,
                         wednesday,
#                          has_software,
#                          has_bing,
#                          has_alice_top,
                         
                        ]
    
    feature_names = ['morning', 'day','evening', 'night',
#                      'durations',
                     'day_of_week',
                     'year_month',
                     'sunday',
                     'start_google',
                     'start_youtube',
                     'fr_domains',
                     'start_vk',
                     'msft_usage', 
                     'summer',
#                      'week',
                     'has_google',
                     'has_youtube',
#                      'in_alice_top',
                     'has_vk',
                     'has_facebook',
                     'may',
                     'october',
                     'wednesday',
#                      'has_software',
#                      'has_bing',
#                      'has_alice_top',
                    ]
        
    X = hstack(objects_to_hstack)
    return X, feature_names

X_train, new_feat_names = add_features(train_times, train_sites, X_train_sites, top_alice_sites)
X_test, _ = add_features(test_times, test_sites, X_test_sites, top_alice_sites)

In [102]:
%%time
cv_scores_tuned_new = train_and_predict(model=logit_grid_searcher.best_estimator_, X_train=X_train, y_train=y_train,
                                         X_test=X_test, 
                                         site_feature_names=vectorizer.get_feature_names(),
                                         new_feature_names=new_feat_names,
                                         cv=time_split,
                                         submission_file_name=pred_path + get_next_filename(filename)[0])

CV scores [0.8545474  0.84723063 0.89421269 0.96940102 0.92497993 0.95356929
 0.93755308 0.94421472 0.96566193 0.96889042]

CV mean: 0.9260261106255274, CV std: 0.043408727168706605


Weight?,Feature
+9.589,www.express.co.uk
+8.492,cid-ed6c3e6a5c6608a4.users.storage.live.com
+6.153,tru.am
+6.141,browser-update.org
+5.580,fr.glee.wikia.com
+5.357,www.info-jeunes.net
+5.308,api.bing.com
+5.218,youwatch.org
+5.158,www.kelbillet.com
+5.095,www.banque-chalus.fr


New feature weights:
          feature      coef
13     has_google -4.640624
12         summer -3.256771
2         evening -3.241522
0         morning -3.096060
18        october -3.054415
6          sunday -1.875893
17            may -1.197759
5      year_month -0.556350
7    start_google -0.541889
4     day_of_week -0.390644
3           night  0.000000
10       start_vk  0.154254
9      fr_domains  0.156098
8   start_youtube  0.157584
11     msft_usage  0.327254
16   has_facebook  0.401129
14    has_youtube  0.504864
1             day  0.751057
19      wednesday  1.154063
15         has_vk  1.742465
Wall time: 22.1 s


In [101]:
cv_scores_tuned_new > cv_scores_tuned

array([False,  True, False, False, False, False, False,  True, False,
        True])

1. with wednesday - 0.95832 on LB
1. with software - 0.95771 on LB
1. with has_bing - 0.96226 on LB

In [93]:
logit_grid_searcher.best_params_

{'C': 2.5118864315095824, 'tol': 0.004}